# Custom Chatbot Notebook

An OpenAI client is initialised by using environment variables and a tokenizer is set up for a specific model (`gpt-4o-mini-2024-07-18`). Also,the necessary libraries and custom utility functions are imported.

In [22]:
import pandas as pd
import os
from pathlib import Path
from dotenv import load_dotenv
import tiktoken

# Custom Functions
from fncs.utilities import (
    create_openai_client,
    response_generator,
    prompt_builder,
    calculate_total_cost
    )
from fncs.retrieval import (
    get_embedding,
    search_text,
    control_chunk_context
    )

# Load environment vars:
load_dotenv()
base_url_voc = os.getenv("OPENAI_BASE_VOC")
api_key_voc = os.getenv("OPENAI_API_VOC")
# Deployment model names
chat_name = 'gpt-4o' # 'gpt-4o-mini-2024-07-18' # 'gpt-4o-mini'
emb_name = 'text-embedding-3-small'
# Initialising OpenAI client
openai_client = create_openai_client(api_key= api_key_voc, base_url= base_url_voc)
tokenizer = tiktoken.encoding_for_model(chat_name)

### Loading dataset

In [23]:
proj_dir = Path(os.getcwd())
df = pd.read_csv(proj_dir / "data" / "2023_fashion_trends_embeddings.csv")
df.head(3)

,text,embeddings
0,Title: 7 Fashion Trends That Will Take Over 20...,"[-0.0008604738395661116, 0.02634955383837223, ..."
1,Title: 7 Fashion Trends That Will Take Over 20...,"[0.01805400848388672, 0.049275610595941544, 0...."
2,Title: 7 Fashion Trends That Will Take Over 20...,"[0.0642574205994606, 0.023316336795687675, -0...."


The embeddings are stored as text/string in the DataFrame and need to be converted to lists/arrays

In [24]:
import ast
# Converting the string representations of embeddings to actual lists
df['embeddings'] = df['embeddings'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

Checking transformation

In [25]:
type(df[['embeddings']].iloc[0].values[0])

list

### Calculating Cosine Distances based on query


Below I create a query string about fashion trends in 2023. Then, by using the `get_embedding` function, the embeddings of the query are generated, by passing the query, OpenAI client, and embedding model as inputs.

In [26]:
query = "What is the most popular fashion trend about pants in 2023?"
query_emb = get_embedding(text=query, client = openai_client, model=emb_name)

The DataFrame `df` is sorted based on the cosine distance between the query embedding (`query_emb`) and the embeddings in the DataFrame using the `search_text` function, and stores the result in `df_sorted`.

In [27]:
df_sorted = search_text(df=df, embs_query=query_emb, cosine='distance')

In [28]:
df_sorted.head()

,text,embeddings,distance
1,Title: 7 Fashion Trends That Will Take Over 20...,"[0.01805400848388672, 0.049275610595941544, 0....",0.328703
58,Title: Spring/Summer 2023 Fashion Trends: 21 E...,"[0.03040272183716297, 0.039167024195194244, 0....",0.386368
3,Title: 7 Fashion Trends That Will Take Over 20...,"[0.030550595372915268, 0.04003358259797096, -0...",0.406843
44,Title: Spring/Summer 2023 Fashion Trends: 21 E...,"[0.022972876206040382, 0.05934659391641617, 0....",0.409787
5,Title: 7 Fashion Trends That Will Take Over 20...,"[0.01975318044424057, 0.05273978039622307, 0.0...",0.427201


### Prompt Template



Creating the system prompt to be used in the chatbot

In [60]:
system_prompt = "You are an expert fashion trend analyser. Based only on the provided information you must analyse and summarise the trends and provide an accurate answer."

print(f"System Prompt Tokens: {len(tokenizer.encode(system_prompt))}")

System Prompt Tokens: 28


Creating the user prompt to be used in the chatbot

In [61]:
user_prompt = \
"""
***Question: {}

***Context:
<--Start of Context-->
{}
<--End of Context-->

**Instructions:
- Answer based ONLY on the provided context above
- Do not include external knowledge
- Be concise and specific

**Required Format:
1. Answer:
   [Your detailed response here]

2. Key Points:
   • [Bullet point 1]
   • [Bullet point 2]
   • [...]

3. Sources:
   • [Source URL 1]
   • [Source URL 2]

Note: If the answer cannot be determined from the provided context,
state: "Cannot be determined from the given context."
"""
print(f"User Prompt Tokens BEFORE context insertion: {len(tokenizer.encode(user_prompt))}")

User Prompt Tokens BEFORE context insertion: 130


In [63]:
# to be used in performance demonstration later
user_prompt_without_context = \
"""
***Question: {}

**Instructions:
- Be concise and specific

**Required Format:
1. Answer:
   [Your detailed response here]

2. Key Points:
   • [Bullet point 1]
   • [Bullet point 2]
   • [...]

3. Sources:
   • [Source URL 1]
   • [Source URL 2]

Note: If the answer cannot be determined from the provided context,
state: "Cannot be determined from the given context."
"""
print(f"User Prompt Tokens BEFORE context insertion: {len(tokenizer.encode(user_prompt))}")

User Prompt Tokens BEFORE context insertion: 130


#### Apply token controller function ( fnc: control_chunk_context )

The variable `max_token_count` to 1000, serves as a limit for the total number of tokens allowed in a prompt.

In [64]:
#parameter that control the prompt tokens:
max_token_count = 1000

The code below calculates the current token count of the prompts (system and user) and generates a context by selecting data from the sorted DataFrame (`df_sorted`) based on a maximum allowed token limit (`max_token_count`) using the `control_chunk_context` function.

In [65]:
current_token_count = len(tokenizer.encode(user_prompt)) + len(tokenizer.encode(system_prompt))
# Create context from sorted dataframe according to the max token limit
context = control_chunk_context(
    df_sorted,
    current_token_count,
    max_token_count,
    tokenizer = tokenizer
)

 Below, the final `user_prompt` is created by inserting the generated `context` into the prompt template and by formatting it with the query and context.

In [66]:
# prompt template params
context_inprompt = "\n----\n".join(context)
user_prompt = user_prompt.format(query, context_inprompt)

print(user_prompt)


***Question: What is the most popular fashion trend about pants in 2023?

***Context:
<--Start of Context-->
Title: 7 Fashion Trends That Will Take Over 2023 — Shop Them Now

2023 Fashion Trend: Cargo Pants. Utilitarian wear is in for 2023, which sets the stage for the return of the cargo pant. But these aren't the shapeless, low-rise pants of the Y2K era. For spring, this trend is translated into tailored silhouettes, interesting pocket placements, elevated fabrics like silk and organza, and colors that go beyond khaki and olive.

Source URL: www.refinery29.com

----
Title: Spring/Summer 2023 Fashion Trends: 21 Expert-Approved Looks You Need to See

Every buyer I have spoken to has been most excited by the many pairs of perfectly cut trousers in the spring/summer 2023 collections, which actually should hardly come as a surprise. It's been the year of the trouser after all, and that looks set to continue as designers have become more and more playful with their pants. From pedal pushe

In [67]:
print(f"User Prompt Tokens AFTER context insertion: {len(tokenizer.encode(user_prompt))}")

User Prompt Tokens AFTER context insertion: 887


### Custom Query Completion

**Finally, the code below generates a final prompt using the `prompt_builder` function by combining the system and user prompts. It then sends the prompt to the OpenAI model (`chat_model`) using the `response_generator` function with specified additional options (e.g., `temperature=0`) to generate an AI response. It also calculates the total cost in EUR based on the API usage (`response_full.usage`) for the specific deployment (`gpt-4o-mini`).**

In [68]:
final_prompt = prompt_builder(system_content= system_prompt, user_content_prompt= user_prompt)
additional_options = {"temperature": 0.4,}
response, response_full = response_generator(openai_client, chat_model=chat_name, prompts=final_prompt, options=additional_options)
cost_eur = calculate_total_cost(response_usage= response_full.usage,
                                deployment_name= chat_name)
print(f'Query Completion Total Cost is: {cost_eur} eur')

Query Completion Total Cost is: 0.00373177956 eur


In [69]:
print(response)

1. Answer:
   The most popular fashion trend about pants in 2023 is the resurgence of cargo pants with a modern twist, featuring tailored silhouettes and elevated fabrics. Additionally, baggy and wide-leg denim styles are also prominent.

2. Key Points:
   • Cargo pants are making a comeback with tailored silhouettes and unique fabrics.
   • Wide-leg and baggy denim styles are significant trends.
   • Designers are experimenting with various pant styles, including pedal pushers and puddle hemlines.

3. Sources:
   • www.refinery29.com
   • www.whowhatwear.com


In [70]:
print('Total Tokens: ', response_full.usage.total_tokens)
print('Total Completion Tokens: ', response_full.usage.completion_tokens)
print('Total Prompt Tokens: ', response_full.usage.prompt_tokens)

Total Tokens:  1048
Total Completion Tokens:  122
Total Prompt Tokens:  926


## Demonstrating Performance